In [ ]:
# Check nvidia-cart
!nvidia-smi

Tue May 25 09:04:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.1.1-bin-hadoop3.2.tgz

spark-3.1.1-bin-hadoop3.2/
spark-3.1.1-bin-hadoop3.2/NOTICE
spark-3.1.1-bin-hadoop3.2/kubernetes/
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/python_executable_check.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/autoscale.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/worker_memory_check.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/py_container_checks.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/decommissioning.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/pyfiles.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/decommissioning_cleanup.py
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/decom.sh
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/bindings/
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/bindings/R/
spark-3.1.1-bin-hadoop3.2/kubernetes/docker

In [ ]:
# 
!ls '/content/'

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
# 
!pip install -q findspark

In [ ]:
# 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2/"

In [ ]:
# Подключаем диск и загружаем dataset
from google.colab import files
files.upload()

from google.colab import drive
drive.mount('/content/drive')

Saving archive.zip to archive.zip
Mounted at /content/drive


In [ ]:
# Проверяем наличие папки. Если есть, удаляем ее
!rm -R /content/date

rm: cannot remove '/content/date': No such file or directory


In [ ]:
# Извлекаем данные
!unzip -q '/content/archive.zip' -d '/content/date'

In [ ]:
# Инциализируем pyspark из директории с библиотекой
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_processing').getOrCreate()

df = spark.read.csv('/content/date/Case.csv', inferSchema=True, header=True)
df.columns

[' case_id',
 'province',
 'city',
 'group',
 'infection_case',
 'confirmed',
 'latitude',
 'longitude']

In [ ]:
df.show()

+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|      Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|        Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|   Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|      Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
| 1000006|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
| 1000007|   Seoul|from other city| true|SMR Newly Planted...|       36|        -|         -|
| 1000008|   Seoul|  Dongdaemun-gu| true|       Dongan Churc

In [ ]:
df.limit(10).toPandas()

,province,city,infection_source,confirmed
0,Seoul,Yongsan-gu,Itaewon Clubs,139
1,Seoul,Gwanak-gu,Richway,119
2,Seoul,Guro-gu,Guro-gu Call Center,95
3,Seoul,Yangcheon-gu,Yangcheon Table Tennis Club,43
4,Seoul,Dobong-gu,Day Care Center,43
5,Seoul,Guro-gu,Manmin Central Church,41
6,Seoul,from other city,SMR Newly Planted Churches Group,36
7,Seoul,Dongdaemun-gu,Dongan Church,17
8,Seoul,from other city,Coupang Logistics Center,25
9,Seoul,Gwanak-gu,Wangsung Church,30


In [ ]:
# 
df = df.withColumnRenamed("infection_case","infection_source")
df.show()

+--------+---------------+--------------------+---------+
|province|           city|    infection_source|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

In [ ]:
df = df.select('province','city','infection_source','confirmed')
df.show()

+--------+---------------+--------------------+---------+
|province|           city|    infection_source|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

In [ ]:
df.sort("confirmed").show()

+-----------------+---------------+--------------------+---------+
|         province|           city|    infection_source|confirmed|
+-----------------+---------------+--------------------+---------+
|          Jeju-do|              -|contact with patient|        0|
|       Gangwon-do|              -|contact with patient|        0|
|            Seoul|     Gangseo-gu|SJ Investment Cal...|        0|
|            Busan|from other city|Cheongdo Daenam H...|        1|
|     Jeollabuk-do|from other city|  Shincheonji Church|        1|
|            Seoul|from other city|Anyang Gunpo Past...|        1|
|            Seoul|     Gangnam-gu|Gangnam Dongin Ch...|        1|
|           Sejong|from other city|  Shincheonji Church|        1|
|     Jeollanam-do|from other city|  Shincheonji Church|        1|
|          Jeju-do|from other city|       Itaewon Clubs|        1|
|            Seoul|from other city|Daejeon door-to-d...|        1|
|            Seoul|              -|         Orange Life|      

In [ ]:
# descending Sort
from pyspark.sql import functions as F
df.sort(F.desc("confirmed")).show()

+-----------------+---------------+--------------------+---------+
|         province|           city|    infection_source|confirmed|
+-----------------+---------------+--------------------+---------+
|            Daegu|         Nam-gu|  Shincheonji Church|     4511|
|            Daegu|              -|contact with patient|      917|
|            Daegu|              -|                 etc|      747|
| Gyeongsangbuk-do|from other city|  Shincheonji Church|      566|
|      Gyeonggi-do|              -|     overseas inflow|      305|
|            Seoul|              -|     overseas inflow|      298|
|            Daegu|   Dalseong-gun|Second Mi-Ju Hosp...|      196|
| Gyeongsangbuk-do|              -|contact with patient|      190|
|            Seoul|              -|contact with patient|      162|
|            Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
| Gyeongsangbuk-do|              -|                 etc|      133|
|            Daegu|         Seo-gu|Hansarang Convale...|      

In [ ]:
# когда Pyspark читает двойной как integer или строки, 
# В таких случаях, вы можете использовать функцию литой для преобразования типов
from pyspark.sql.types import DoubleType, IntegerType, StringType

df = df.withColumn('confirmed', F.col('confirmed').cast(IntegerType()))
df = df.withColumn('city', F.col('city').cast(StringType()))
df.show()

+--------+---------------+--------------------+---------+
|province|           city|    infection_source|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

In [ ]:
# фильтровать рамку данных с помощью нескольких условий, 
# используя условия AND (), OR (|) и NOT

df.filter((df.confirmed>10) & (df.province=='Daegu')).show()

+--------+------------+--------------------+---------+
|province|        city|    infection_source|confirmed|
+--------+------------+--------------------+---------+
|   Daegu|      Nam-gu|  Shincheonji Church|     4511|
|   Daegu|Dalseong-gun|Second Mi-Ju Hosp...|      196|
|   Daegu|      Seo-gu|Hansarang Convale...|      124|
|   Daegu|Dalseong-gun|Daesil Convalesce...|      101|
|   Daegu|     Dong-gu|     Fatima Hospital|       39|
|   Daegu|           -|     overseas inflow|       41|
|   Daegu|           -|contact with patient|      917|
|   Daegu|           -|                 etc|      747|
+--------+------------+--------------------+---------+



In [ ]:
# groupBy
from pyspark.sql import functions as F

df.groupBy(["province","city"]).agg(F.sum("confirmed") ,F.max("confirmed")).show()

+----------------+---------------+--------------+--------------+
|        province|           city|sum(confirmed)|max(confirmed)|
+----------------+---------------+--------------+--------------+
|Gyeongsangnam-do|       Jinju-si|             9|             9|
|           Seoul|        Guro-gu|           139|            95|
|           Seoul|     Gangnam-gu|            18|             7|
|         Daejeon|              -|           100|            55|
|    Jeollabuk-do|from other city|             6|             3|
|Gyeongsangnam-do|Changnyeong-gun|             7|             7|
|           Seoul|              -|           561|           298|
|         Jeju-do|from other city|             1|             1|
|Gyeongsangbuk-do|              -|           345|           190|
|Gyeongsangnam-do|   Geochang-gun|            18|            10|
|Gyeongsangbuk-do|        Gumi-si|            10|            10|
|         Incheon|from other city|           117|            53|
|           Busan|       

In [ ]:
# использовать ключевое слово для переименования столбцов в самой команде.aliasagg

df.groupBy(["province","city"]).agg(
    F.sum("confirmed").alias("TotalConfirmed"),\
    F.max("confirmed").alias("MaxFromOneConfirmedCase")\
    ).show()

+----------------+---------------+--------------+-----------------------+
|        province|           city|TotalConfirmed|MaxFromOneConfirmedCase|
+----------------+---------------+--------------+-----------------------+
|Gyeongsangnam-do|       Jinju-si|             9|                      9|
|           Seoul|        Guro-gu|           139|                     95|
|           Seoul|     Gangnam-gu|            18|                      7|
|         Daejeon|              -|           100|                     55|
|    Jeollabuk-do|from other city|             6|                      3|
|Gyeongsangnam-do|Changnyeong-gun|             7|                      7|
|           Seoul|              -|           561|                    298|
|         Jeju-do|from other city|             1|                      1|
|Gyeongsangbuk-do|              -|           345|                    190|
|Gyeongsangnam-do|   Geochang-gun|            18|                     10|
|Gyeongsangbuk-do|        Gumi-si|    

In [ ]:
# Join

regions = spark.read.load("/content/date/Region.csv",format="csv", sep=",", inferSchema="true", header="true")
regions.limit(10).toPandas()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080
5,10050,Seoul,Gwanak-gu,37.478290,126.951502,22,33,1,0.89,15.12,4.9,909
6,10060,Seoul,Gwangjin-gu,37.538712,127.082366,22,33,3,1.16,13.75,4.8,723
7,10070,Seoul,Guro-gu,37.495632,126.887650,26,34,3,1.00,16.21,5.7,741
8,10080,Seoul,Geumcheon-gu,37.456852,126.895229,18,19,0,0.96,16.15,6.7,475
9,10090,Seoul,Nowon-gu,37.654259,127.056294,42,66,6,1.39,15.40,7.4,952


In [ ]:
# Left join

cases = df.join(regions, ['province','city'], how='left')
cases.limit(10).toPandas()

,province,city,infection_source,confirmed,code,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,Seoul,Yongsan-gu,Itaewon Clubs,139,10210.0,37.532768,126.990021,15.0,13.0,1.0,0.68,16.87,6.5,435.0
1,Seoul,Gwanak-gu,Richway,119,10050.0,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0
2,Seoul,Guro-gu,Guro-gu Call Center,95,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
3,Seoul,Yangcheon-gu,Yangcheon Table Tennis Club,43,10190.0,37.517189,126.866618,30.0,43.0,0.0,2.26,13.55,5.5,816.0
4,Seoul,Dobong-gu,Day Care Center,43,10100.0,37.668952,127.047082,23.0,26.0,1.0,0.95,17.89,7.2,485.0
5,Seoul,Guro-gu,Manmin Central Church,41,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
6,Seoul,from other city,SMR Newly Planted Churches Group,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Seoul,Dongdaemun-gu,Dongan Church,17,10110.0,37.574552,127.039721,21.0,31.0,4.0,1.06,17.26,6.7,832.0
8,Seoul,from other city,Coupang Logistics Center,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Seoul,Gwanak-gu,Wangsung Church,30,10050.0,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0


In [ ]:
# Две разные по размеру таблицы

from pyspark.sql.functions import broadcast
cases = cases.join(broadcast(regions), ['province','city'], how='left')
cases.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|province|           city|    infection_source|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|kindergar

In [ ]:
# Broadcast/Map Side Joins
import pandas as pd 

big = pd.DataFrame({'A':[1,1,1,1,2,2,2,2],'price': [i for i in range(8)]}) 
small = pd.DataFrame({'A':[1,1,1,1,2,2],'agg':['sum','mean','max','min','sum','mean']})

big

,A,price
0,1,0
1,1,1
2,1,2
3,1,3
4,2,4
5,2,5
6,2,6
7,2,7


In [ ]:
small

,A,agg
0,1,sum
1,1,mean
2,1,max
3,1,min
4,2,sum
5,2,mean


In [ ]:
big.merge(small,on = ['A'],how='left')

,A,price,agg
0,1,0,sum
1,1,0,mean
2,1,0,max
3,1,0,min
4,1,1,sum
5,1,1,mean
6,1,1,max
7,1,1,min
8,1,2,sum
9,1,2,mean


In [ ]:
# SQL
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

cases.registerTempTable('cases_table')
newDF = sqlContext.sql('select * from cases_table where confirmed>100')
newDF.show()

+-----------------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|         province|           city|    infection_source|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_scho

In [ ]:
# Add columns

import pyspark.sql.functions as F
casesWithNewConfirmed = cases.withColumn("NewConfirmed", 100 + F.col("confirmed"))
casesWithNewConfirmed.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+------------+
|province|           city|    infection_source|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_co

In [ ]:
# 

casesWithExpConfirmed = cases.withColumn("ExpConfirmed", F.exp("confirmed"))
casesWithExpConfirmed.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------+
|province|           city|    infection_source|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_s

In [ ]:
# Использование UDFs Spark

import pyspark.sql.functions as F
from pyspark.sql.types import *
def casesHighLow(confirmed):
    if confirmed < 50: 
        return 'low'
    else:
        return 'high'
    
#convert to a UDF Function by passing in the function and return type of function
casesHighLowUDF = F.udf(casesHighLow, StringType())
CasesWithHighLow = cases.withColumn("HighLow", casesHighLowUDF("confirmed"))
CasesWithHighLow.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-------+
|province|           city|    infection_source|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| code| latitude| longitude|elementary_school_count|k

In [ ]:
df = spark.read.csv('/content/date/Case.csv', inferSchema=True, header=True)
df.columns

In [ ]:
# Spark UDFs
import pyspark.sql.functions as F
from pyspark.sql.types import *

def casesHighLow(confirmed):
    if confirmed < 50: 
        return 'low'
    else:
        return 'high'
    
#convert to a UDF Function by passing in the function and return type of function
casesHighLowUDF = F.udf(casesHighLow, StringType())

CasesWithHighLow = df.withColumn("HighLow", casesHighLowUDF("confirmed"))
CasesWithHighLow.show()

+--------+---------------+--------------------+---------+-------+
|province|           city|    infection_source|confirmed|HighLow|
+--------+---------------+--------------------+---------+-------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|   high|
|   Seoul|      Gwanak-gu|             Richway|      119|   high|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|   high|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|    low|
|   Seoul|      Dobong-gu|     Day Care Center|       43|    low|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|    low|
|   Seoul|from other city|SMR Newly Planted...|       36|    low|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|    low|
|   Seoul|from other city|Coupang Logistics...|       25|    low|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|    low|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|    low|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|    low|
|   Seoul|

In [ ]:
# 

import math
from pyspark.sql import Row
def rowwise_function(row):
    # convert row to python dictionary:
    row_dict = row.asDict()
    # Add a new key in the dictionary with the new column name and value.
    # This might be a big complex function.
    row_dict['expConfirmed'] = float(np.exp(row_dict['confirmed']))
    # convert dict to row back again:
    newrow = Row(**row_dict)
    # return new row
    return newrow

# convert cases dataframe to RDD
cases_rdd = df.rdd

# apply our function to RDD
cases_rdd_new = cases_rdd.map(lambda row: rowwise_function(row))

# Convert RDD Back to DataFrame
casesNewDf = sqlContext.createDataFrame(cases_rdd_new)

casesNewDf.show()

Py4JJavaError: ignored

In [ ]:
df.printSchema()

root
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_source: string (nullable = true)
 |-- confirmed: integer (nullable = true)



In [ ]:
from pyspark.sql.types import IntegerType, StringType, DoubleType, BooleanType
from pyspark.sql.types import StructType, StructField

# Declare the schema for the output of our function

outSchema = StructType([StructField('case_id',IntegerType(),True),
                        StructField('province',StringType(),True),
                        StructField('city',StringType(),True),
                        StructField('group',BooleanType(),True),
                        StructField('infection_case',StringType(),True),
                        StructField('confirmed',IntegerType(),True),
                        StructField('latitude',StringType(),True),
                        StructField('longitude',StringType(),True),
                        StructField('normalized_confirmed',DoubleType(),True)
                       ])
# decorate our function with pandas_udf decorator
@F.pandas_udf(outSchema, F.PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    v = pdf.confirmed
    v = v - v.mean()
    pdf['normalized_confirmed'] = v
    return pdf

confirmed_groupwise_normalization =df.groupby("infection_source").apply(subtract_mean)


/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


In [ ]:
# Spark Window Functions

timeprovince = spark.read.load("/content/date/TimeProvince.csv",format="csv", \
                        sep=",", inferSchema="true", header="true")
timeprovince.show()

+----------+----+-----------------+---------+--------+--------+
|      date|time|         province|confirmed|released|deceased|
+----------+----+-----------------+---------+--------+--------+
|2020-01-20|  16|            Seoul|        0|       0|       0|
|2020-01-20|  16|            Busan|        0|       0|       0|
|2020-01-20|  16|            Daegu|        0|       0|       0|
|2020-01-20|  16|          Incheon|        1|       0|       0|
|2020-01-20|  16|          Gwangju|        0|       0|       0|
|2020-01-20|  16|          Daejeon|        0|       0|       0|
|2020-01-20|  16|            Ulsan|        0|       0|       0|
|2020-01-20|  16|           Sejong|        0|       0|       0|
|2020-01-20|  16|      Gyeonggi-do|        0|       0|       0|
|2020-01-20|  16|       Gangwon-do|        0|       0|       0|
|2020-01-20|  16|Chungcheongbuk-do|        0|       0|       0|
|2020-01-20|  16|Chungcheongnam-do|        0|       0|       0|
|2020-01-20|  16|     Jeollabuk-do|     

In [ ]:
# Ranking

from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['province']).orderBy(F.desc('confirmed'))
df.withColumn("rank",F.rank().over(windowSpec)).show()

+-----------------+---------------+--------------------+---------+----+
|         province|           city|    infection_source|confirmed|rank|
+-----------------+---------------+--------------------+---------+----+
|           Sejong|         Sejong|Ministry of Ocean...|       31|   1|
|           Sejong|         Sejong|gym facility in S...|        8|   2|
|           Sejong|              -|     overseas inflow|        5|   3|
|           Sejong|              -|contact with patient|        3|   4|
|           Sejong|from other city|  Shincheonji Church|        1|   5|
|           Sejong|              -|                 etc|        1|   5|
|            Ulsan|              -|     overseas inflow|       25|   1|
|            Ulsan|from other city|  Shincheonji Church|       16|   2|
|            Ulsan|              -|                 etc|        7|   3|
|            Ulsan|              -|contact with patient|        3|   4|
|Chungcheongbuk-do|              -|     overseas inflow|       1

In [ ]:
# Lag


from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['province']).orderBy('date')
timeprovinceWithLag = timeprovince.withColumn("lag_7",F.lag("confirmed", 7).over(windowSpec))

timeprovinceWithLag.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+-----+
|      date|time|province|confirmed|released|deceased|lag_7|
+----------+----+--------+---------+--------+--------+-----+
|2020-03-11|   0|  Sejong|       10|       0|       0|    1|
|2020-03-12|   0|  Sejong|       15|       0|       0|    1|
|2020-03-13|   0|  Sejong|       32|       0|       0|    1|
|2020-03-14|   0|  Sejong|       38|       0|       0|    2|
|2020-03-15|   0|  Sejong|       39|       0|       0|    3|
|2020-03-16|   0|  Sejong|       40|       0|       0|    6|
|2020-03-17|   0|  Sejong|       40|       0|       0|    8|
|2020-03-18|   0|  Sejong|       41|       0|       0|   10|
|2020-03-19|   0|  Sejong|       41|       0|       0|   15|
|2020-03-20|   0|  Sejong|       41|       0|       0|   32|
|2020-03-21|   0|  Sejong|       41|       2|       0|   38|
|2020-03-22|   0|  Sejong|       41|       3|       0|   39|
|2020-03-23|   0|  Sejong|       42|       3|       0|   40|
|2020-03-24|   0|  Sejon

In [ ]:
# Rolling Aggregations

from pyspark.sql.window import Window

windowSpec = Window().partitionBy(['province']).orderBy('date').rowsBetween(-6,0)
timeprovinceWithRoll = timeprovince.withColumn("roll_7_confirmed",F.mean("confirmed").over(windowSpec))
timeprovinceWithRoll.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+------------------+
|      date|time|province|confirmed|released|deceased|  roll_7_confirmed|
+----------+----+--------+---------+--------+--------+------------------+
|2020-03-11|   0|  Sejong|       10|       0|       0| 4.428571428571429|
|2020-03-12|   0|  Sejong|       15|       0|       0| 6.428571428571429|
|2020-03-13|   0|  Sejong|       32|       0|       0|10.857142857142858|
|2020-03-14|   0|  Sejong|       38|       0|       0|              16.0|
|2020-03-15|   0|  Sejong|       39|       0|       0|21.142857142857142|
|2020-03-16|   0|  Sejong|       40|       0|       0|              26.0|
|2020-03-17|   0|  Sejong|       40|       0|       0|30.571428571428573|
|2020-03-18|   0|  Sejong|       41|       0|       0|              35.0|
|2020-03-19|   0|  Sejong|       41|       0|       0|38.714285714285715|
|2020-03-20|   0|  Sejong|       41|       0|       0|              40.0|
|2020-03-21|   0|  Sejong|       41|  

In [ ]:
# Running Totals

from pyspark.sql.window import Window

windowSpec = Window().partitionBy(['province']).orderBy('date').rowsBetween(Window.unboundedPreceding,Window.currentRow)
timeprovinceWithRoll = timeprovince.withColumn("cumulative_confirmed",F.sum("confirmed").over(windowSpec))
timeprovinceWithRoll.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+--------------------+
|      date|time|province|confirmed|released|deceased|cumulative_confirmed|
+----------+----+--------+---------+--------+--------+--------------------+
|2020-03-11|   0|  Sejong|       10|       0|       0|                  43|
|2020-03-12|   0|  Sejong|       15|       0|       0|                  58|
|2020-03-13|   0|  Sejong|       32|       0|       0|                  90|
|2020-03-14|   0|  Sejong|       38|       0|       0|                 128|
|2020-03-15|   0|  Sejong|       39|       0|       0|                 167|
|2020-03-16|   0|  Sejong|       40|       0|       0|                 207|
|2020-03-17|   0|  Sejong|       40|       0|       0|                 247|
|2020-03-18|   0|  Sejong|       41|       0|       0|                 288|
|2020-03-19|   0|  Sejong|       41|       0|       0|                 329|
|2020-03-20|   0|  Sejong|       41|       0|       0|                 370|
|2020-03-21|

In [ ]:
# Pivot Dataframes

pivotedTimeprovince = timeprovince.groupBy('date').pivot('province') \
                      .agg(F.sum('confirmed').alias('confirmed') , F.sum('released').alias('released'))
pivotedTimeprovince.limit(10).toPandas()

,date,Busan_confirmed,Busan_released,Chungcheongbuk-do_confirmed,Chungcheongbuk-do_released,Chungcheongnam-do_confirmed,Chungcheongnam-do_released,Daegu_confirmed,Daegu_released,Daejeon_confirmed,Daejeon_released,Gangwon-do_confirmed,Gangwon-do_released,Gwangju_confirmed,Gwangju_released,Gyeonggi-do_confirmed,Gyeonggi-do_released,Gyeongsangbuk-do_confirmed,Gyeongsangbuk-do_released,Gyeongsangnam-do_confirmed,Gyeongsangnam-do_released,Incheon_confirmed,Incheon_released,Jeju-do_confirmed,Jeju-do_released,Jeollabuk-do_confirmed,Jeollabuk-do_released,Jeollanam-do_confirmed,Jeollanam-do_released,Sejong_confirmed,Sejong_released,Seoul_confirmed,Seoul_released,Ulsan_confirmed,Ulsan_released
0,2020-04-13,126,103,45,31,139,109,6819,5395,39,23,49,28,27,19,631,305,1337,1020,115,84,87,39,12,5,17,8,15,6,46,22,610,214,41,33
1,2020-02-26,58,0,5,0,3,0,710,1,5,0,6,0,9,2,51,8,317,1,34,0,3,1,2,0,3,1,1,0,1,0,49,8,4,0
2,2020-06-24,152,142,62,56,162,146,6903,6687,94,44,63,52,33,32,1137,761,1386,1321,133,124,333,174,19,15,25,20,20,18,49,47,1241,747,55,48
3,2020-06-08,147,141,61,49,148,143,6888,6641,46,43,58,51,32,30,942,680,1383,1309,124,122,283,118,15,13,21,19,20,17,47,47,996,651,53,46
4,2020-06-22,150,142,61,56,161,144,6900,6681,82,44,62,52,33,32,1123,753,1385,1320,133,124,329,169,19,15,24,20,20,18,49,47,1224,733,53,48
5,2020-06-20,150,142,61,55,158,144,6898,6679,72,44,60,52,32,32,1107,741,1384,1319,132,124,328,165,19,15,23,20,20,18,48,47,1202,730,53,48
6,2020-04-12,126,103,45,29,139,108,6816,5356,39,23,49,27,27,19,628,292,1333,1013,115,84,86,38,12,4,17,8,15,5,46,22,602,202,41,32
7,2020-02-13,0,0,0,0,0,0,0,0,0,0,0,0,1,0,12,5,0,1,0,0,1,1,0,0,1,1,0,0,0,0,14,2,0,0
8,2020-04-20,132,115,45,38,141,122,6833,5769,39,26,53,28,30,22,656,387,1361,1064,116,87,92,56,13,6,17,9,15,8,46,29,624,304,43,34
9,2020-06-09,147,141,61,49,150,143,6888,6643,46,43,58,51,32,30,955,680,1383,1310,125,123,286,121,15,13,21,19,20,17,47,47,1015,664,53,46


In [ ]:
# Unpivot/Stack Dataframes

pivotedTimeprovince.columns

['date',
 'Busan_confirmed',
 'Busan_released',
 'Chungcheongbuk-do_confirmed',
 'Chungcheongbuk-do_released',
 'Chungcheongnam-do_confirmed',
 'Chungcheongnam-do_released',
 'Daegu_confirmed',
 'Daegu_released',
 'Daejeon_confirmed',
 'Daejeon_released',
 'Gangwon-do_confirmed',
 'Gangwon-do_released',
 'Gwangju_confirmed',
 'Gwangju_released',
 'Gyeonggi-do_confirmed',
 'Gyeonggi-do_released',
 'Gyeongsangbuk-do_confirmed',
 'Gyeongsangbuk-do_released',
 'Gyeongsangnam-do_confirmed',
 'Gyeongsangnam-do_released',
 'Incheon_confirmed',
 'Incheon_released',
 'Jeju-do_confirmed',
 'Jeju-do_released',
 'Jeollabuk-do_confirmed',
 'Jeollabuk-do_released',
 'Jeollanam-do_confirmed',
 'Jeollanam-do_released',
 'Sejong_confirmed',
 'Sejong_released',
 'Seoul_confirmed',
 'Seoul_released',
 'Ulsan_confirmed',
 'Ulsan_released']

In [ ]:
newColnames = [x.replace("-","_") for x in pivotedTimeprovince.columns]

In [ ]:
pivotedTimeprovince = pivotedTimeprovince.toDF(*newColnames)

In [ ]:
expression = ""
cnt=0
for column in pivotedTimeprovince.columns:
    if column!='date':
        cnt +=1
        expression += f"'{column}' , {column},"
        
expression = f"stack({cnt}, {expression[:-1]}) as (Type,Value)"

In [ ]:
unpivotedTimeprovince = pivotedTimeprovince.select('date',F.expr(expression))
unpivotedTimeprovince.show()

+----------+--------------------+-----+
|      date|                Type|Value|
+----------+--------------------+-----+
|2020-04-13|     Busan_confirmed|  126|
|2020-04-13|      Busan_released|  103|
|2020-04-13|Chungcheongbuk_do...|   45|
|2020-04-13|Chungcheongbuk_do...|   31|
|2020-04-13|Chungcheongnam_do...|  139|
|2020-04-13|Chungcheongnam_do...|  109|
|2020-04-13|     Daegu_confirmed| 6819|
|2020-04-13|      Daegu_released| 5395|
|2020-04-13|   Daejeon_confirmed|   39|
|2020-04-13|    Daejeon_released|   23|
|2020-04-13|Gangwon_do_confirmed|   49|
|2020-04-13| Gangwon_do_released|   28|
|2020-04-13|   Gwangju_confirmed|   27|
|2020-04-13|    Gwangju_released|   19|
|2020-04-13|Gyeonggi_do_confi...|  631|
|2020-04-13|Gyeonggi_do_released|  305|
|2020-04-13|Gyeongsangbuk_do_...| 1337|
|2020-04-13|Gyeongsangbuk_do_...| 1020|
|2020-04-13|Gyeongsangnam_do_...|  115|
|2020-04-13|Gyeongsangnam_do_...|   84|
+----------+--------------------+-----+
only showing top 20 rows

